In [29]:
import numpy as np
import pandas as pd

In [30]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [31]:
df = pd.read_csv('covid_toy.csv')

In [32]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


## Without Column Transformer

### We have to convert ordinal and nominal data differently. That is hectic

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [34]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [35]:
# adding simple imputer to fever column
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [36]:
# Ordinal Encoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [37]:
# One Hot Encoding -> Gender, City
ohe = OneHotEncoder(drop='first')
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']]).toarray()

x_test_gender_city = ohe.fit_transform(x_test[['gender','city']]).toarray()

In [38]:
# extracting age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [40]:
train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

train_transformed.shape

(80, 7)

## With Column Transformer

In [41]:
from sklearn.compose import ColumnTransformer

In [42]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')

In [43]:
transformer.fit_transform(x_train)

array([[ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  19.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  54.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  51.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.    

In [44]:
transformer.transform(x_test)

array([[ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  11.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [101.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  51.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  64.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  13.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  70.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.    